# В этом ноутбуке 
- я изучаю содержимое тестового файла 
- и собираю соответствующие данные с сайта auto.ru в два приёма:

    -- сначала собираю ссылки на предложения в файл collected_links_May_10;
    
    -- а потом считываю информацию с каждого предложения по ссылке и сохраняю в файл.
    
Из-за того, что связь прерывалась, данные собрались в два файла, которые я потом соединяю в следующем ноутбуке: 
train_p5_full.csv и train_p5_full_SMW.csv

In [1]:
import pandas as pd
pd.set_option("expand_frame_repr", True)
import numpy as np
import re
import time
from datetime import datetime
import json
from fake_useragent import UserAgent
from tqdm.notebook import tqdm
#UserAgent().chrome

#import socks
#import socket
#socks.set_default_proxy(socks.SOCKS5, "localhost", 9150)            # запросы будут посылаться из-под запущенного тора
#socket.socket = socks.socksocket

from bs4 import BeautifulSoup as bs
import requests

In [2]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)


def load_data(title):
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [3]:
def get_unique_values(non_un_list):
    return list(set(non_un_list))
   

# Исследование тестового файла

In [4]:
test_df=pd.read_csv('test.csv')

In [5]:
test_df.columns

Index(['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня'],
      dtype='object')

In [6]:
test_df['brand']=test_df['brand'].apply(lambda x: x.replace('MERCEDES', 'MERCEDES-BENZ')) ## НАДО НА БУДУЩЕЕ!

In [7]:
brands=get_unique_values(test_df['brand'])
display(brands)

['LEXUS',
 'TOYOTA',
 'VOLKSWAGEN',
 'HONDA',
 'MERCEDES-BENZ',
 'AUDI',
 'INFINITI',
 'NISSAN',
 'BMW',
 'MITSUBISHI',
 'SKODA',
 'VOLVO']

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [9]:
working_columns=[ 'brand', 'model_name','modelDate','bodyType','color','engineDisplacement', 
                 'numberOfDoors','vehicleTransmission', 'Привод', 'Руль', 
                 'enginePower','fuelType', 'productionDate', 'mileage',  
                 'Владельцы', 'ПТС','parsing_unixtime']

In [10]:
df=test_df[working_columns]
display(df.sample(3))

,brand,model_name,modelDate,bodyType,color,engineDisplacement,numberOfDoors,vehicleTransmission,Привод,Руль,enginePower,fuelType,productionDate,mileage,Владельцы,ПТС,parsing_unixtime
34230,MERCEDES-BENZ,GLA_CLASS,2017,внедорожник 5 дв.,серый,1.6 LTR,5,роботизированная,передний,Левый,150 N12,бензин,2019,12906,1 владелец,Оригинал,1603671234
29274,BMW,3ER,2011,лифтбек,синий,2.0 LTR,5,автоматическая,полный,Левый,245 N12,бензин,2013,110365,1 владелец,Оригинал,1603559437
12466,NISSAN,LEAF,2010,хэтчбек 5 дв.,чёрный,LTR,5,автоматическая,передний,Правый,109 N12,электро,2016,52000,1 владелец,Оригинал,1603274762


In [11]:
#TIME
# Как показало исследование, выгрузка тестового сета произошла в один день

timestamps_list=get_unique_values(df['parsing_unixtime'])

time_list=[]

timestamp_list=[]

for stamp in timestamps_list:

    ts = int(stamp)

    time_list.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))    

    timestamp_list.append(ts)

mean_date=datetime.utcfromtimestamp(np.array(timestamp_list).mean()).strftime('%Y-%m-%d %H:%M:%S')

print(mean_date)   # и этот день 21 октября 2020

2020-10-21 13:25:33


# Давай парсить

In [12]:
from bs4 import BeautifulSoup as bs
import requests

In [13]:
r = requests.get("https://auto.ru/")

# Convert to a beautiful soup object
soup = bs(r.content)

# Print out the HTML
contents = soup.prettify()
#print(contents)

In [14]:
models_list = soup.find_all(class_='IndexMarks__col') #Главная страница сайта
print(type(models_list))

<class 'bs4.element.ResultSet'>


for row in models_list:
    print(row.prettify())

In [15]:
indexMarks__items=[]                  # Поле, где собраны списки представленных марок
cars=[]
for row in models_list:
    indexMarks__items.append(row.find_all(class_='IndexMarks__item'))
    
    for item in indexMarks__items:
        for car in item:
            cars.append(car)
                      
print(len(cars))                          # Всего  представлено марок на сайте auto.ru

104


In [16]:
car_dict={}                           # Соберём сюда ссылки на представленные марки машин

for car in cars:                          
    value=car['href']
    key=car.find(class_="IndexMarks__item-name").text.upper()
    car_dict[key]=value


In [17]:
for_further_parsing={}                # Выберем только нужные нам марки машин

for brand in brands:
    key=brand
    value=car_dict[key].replace('all','used')   #Скорректируем ссылку, нам нужны подержанные машины
    value=value.replace('cars/','moskva/cars/')   #иначе соскакивает на первую страницу
    for_further_parsing[key]=value
print(for_further_parsing)    

{'LEXUS': 'https://auto.ru/moskva/cars/lexus/used/', 'TOYOTA': 'https://auto.ru/moskva/cars/toyota/used/', 'VOLKSWAGEN': 'https://auto.ru/moskva/cars/volkswagen/used/', 'HONDA': 'https://auto.ru/moskva/cars/honda/used/', 'MERCEDES-BENZ': 'https://auto.ru/moskva/cars/mercedes/used/', 'AUDI': 'https://auto.ru/moskva/cars/audi/used/', 'INFINITI': 'https://auto.ru/moskva/cars/infiniti/used/', 'NISSAN': 'https://auto.ru/moskva/cars/nissan/used/', 'BMW': 'https://auto.ru/moskva/cars/bmw/used/', 'MITSUBISHI': 'https://auto.ru/moskva/cars/mitsubishi/used/', 'SKODA': 'https://auto.ru/moskva/cars/skoda/used/', 'VOLVO': 'https://auto.ru/moskva/cars/volvo/used/'}


## Переходим к маркам

In [18]:
offers_by_brand={}            # Cловарь, куда будут сгружаться линки предложений для каждого брена

for brand in brands:
    offers_by_brand[brand]=[]
print(offers_by_brand)

{'LEXUS': [], 'TOYOTA': [], 'VOLKSWAGEN': [], 'HONDA': [], 'MERCEDES-BENZ': [], 'AUDI': [], 'INFINITI': [], 'NISSAN': [], 'BMW': [], 'MITSUBISHI': [], 'SKODA': [], 'VOLVO': []}


In [19]:
def page_offers(link):     #Cгружает линки предложений со страницы
    offers_list=[]
    
    car_brand = requests.get(link,headers={'User-Agent': UserAgent().chrome})       #текущая страница
    soup_by_brand = bs(car_brand.content)
    
    listing = soup_by_brand.find_all(class_='Link ListingItemTitle-module__link')
    for offer in listing:
        offer_link=offer['href']
        offers_list.append(offer_link)
        
    return offers_list

In [20]:
def all_offers(link): #Сгружает линки со всех страниц
    
    first_page_link=link
    i=1
    count_offers=0
    total_offers_brand=[]
    
    car_brand = requests.get(link,headers={'User-Agent': UserAgent().chrome})
    soup_by_brand = bs(car_brand.content)

    # ВСЕГО ПРЕДЛОЖЕНИЙ ПО ДАННОЙ МАРКЕ (cобираем на первой странице)
    total_offers = soup_by_brand.find(class_='ListingSearchSave-module__count').get_text(strip=True)
    n_off=re.findall('[0-9]+', total_offers)
    t_number=int(''.join(n_off))
    print(t_number)                   #Вывод для контроля
    
    page_1_offers = page_offers(link)                      # считываем первую страницу, вызов функции page_offers
    count_offers+=len(page_1_offers)                       # подсчитываем количество собранных сыылок с предложениями
    total_offers_brand= total_offers_brand + page_1_offers # Добавляем ссылки с первой страницы в список
    
    stop=5000
    if t_number<5000:
        stop=t_number
    
    while count_offers < stop:  #для последующих страниц
        i+=1                        #листаем страницы
        link=first_page_link+'?page='+str(i)
        print(link)
        next_page_offers = page_offers(link)                    # считываем  страницу,вызов функции page_offers
        total_offers_brand= total_offers_brand+next_page_offers # Добавляем ссылки со следующей страницы в список
        count_offers+=len(next_page_offers)                     # подсчитываем количество собранных сыылок с предложениями
        
    return(total_offers_brand)
    

## Action1

In [21]:
for_further_parsing['VOLVO']  #Тест

'https://auto.ru/moskva/cars/volvo/used/'

In [179]:
for brand in brands:          # Cобираем линки предложений со всех страниц по каждому бренду
    
    offers_by_brand.update({brand:all_offers(for_further_parsing[brand])})
    
    print(brand, len(offers_by_brand[brand]))    # Подсчитываем количество собранных предложений

5251
https://auto.ru/moskva/cars/mercedes/used/?page=2
https://auto.ru/moskva/cars/mercedes/used/?page=3
https://auto.ru/moskva/cars/mercedes/used/?page=4
https://auto.ru/moskva/cars/mercedes/used/?page=5
https://auto.ru/moskva/cars/mercedes/used/?page=6
https://auto.ru/moskva/cars/mercedes/used/?page=7
https://auto.ru/moskva/cars/mercedes/used/?page=8
https://auto.ru/moskva/cars/mercedes/used/?page=9
https://auto.ru/moskva/cars/mercedes/used/?page=10
https://auto.ru/moskva/cars/mercedes/used/?page=11
https://auto.ru/moskva/cars/mercedes/used/?page=12
https://auto.ru/moskva/cars/mercedes/used/?page=13
https://auto.ru/moskva/cars/mercedes/used/?page=14
https://auto.ru/moskva/cars/mercedes/used/?page=15
https://auto.ru/moskva/cars/mercedes/used/?page=16
https://auto.ru/moskva/cars/mercedes/used/?page=17
https://auto.ru/moskva/cars/mercedes/used/?page=18
https://auto.ru/moskva/cars/mercedes/used/?page=19
https://auto.ru/moskva/cars/mercedes/used/?page=20
https://auto.ru/moskva/cars/merced

https://auto.ru/moskva/cars/volvo/used/?page=30
https://auto.ru/moskva/cars/volvo/used/?page=31
VOLVO 1129
985
https://auto.ru/moskva/cars/lexus/used/?page=2
https://auto.ru/moskva/cars/lexus/used/?page=3
https://auto.ru/moskva/cars/lexus/used/?page=4
https://auto.ru/moskva/cars/lexus/used/?page=5
https://auto.ru/moskva/cars/lexus/used/?page=6
https://auto.ru/moskva/cars/lexus/used/?page=7
https://auto.ru/moskva/cars/lexus/used/?page=8
https://auto.ru/moskva/cars/lexus/used/?page=9
https://auto.ru/moskva/cars/lexus/used/?page=10
https://auto.ru/moskva/cars/lexus/used/?page=11
https://auto.ru/moskva/cars/lexus/used/?page=12
https://auto.ru/moskva/cars/lexus/used/?page=13
https://auto.ru/moskva/cars/lexus/used/?page=14
https://auto.ru/moskva/cars/lexus/used/?page=15
https://auto.ru/moskva/cars/lexus/used/?page=16
https://auto.ru/moskva/cars/lexus/used/?page=17
https://auto.ru/moskva/cars/lexus/used/?page=18
https://auto.ru/moskva/cars/lexus/used/?page=19
https://auto.ru/moskva/cars/lexus

https://auto.ru/moskva/cars/nissan/used/?page=18
https://auto.ru/moskva/cars/nissan/used/?page=19
https://auto.ru/moskva/cars/nissan/used/?page=20
https://auto.ru/moskva/cars/nissan/used/?page=21
https://auto.ru/moskva/cars/nissan/used/?page=22
https://auto.ru/moskva/cars/nissan/used/?page=23
https://auto.ru/moskva/cars/nissan/used/?page=24
https://auto.ru/moskva/cars/nissan/used/?page=25
https://auto.ru/moskva/cars/nissan/used/?page=26
https://auto.ru/moskva/cars/nissan/used/?page=27
https://auto.ru/moskva/cars/nissan/used/?page=28
https://auto.ru/moskva/cars/nissan/used/?page=29
https://auto.ru/moskva/cars/nissan/used/?page=30
https://auto.ru/moskva/cars/nissan/used/?page=31
https://auto.ru/moskva/cars/nissan/used/?page=32
https://auto.ru/moskva/cars/nissan/used/?page=33
https://auto.ru/moskva/cars/nissan/used/?page=34
https://auto.ru/moskva/cars/nissan/used/?page=35
https://auto.ru/moskva/cars/nissan/used/?page=36
https://auto.ru/moskva/cars/nissan/used/?page=37
https://auto.ru/mosk

https://auto.ru/moskva/cars/bmw/used/?page=98
https://auto.ru/moskva/cars/bmw/used/?page=99
https://auto.ru/moskva/cars/bmw/used/?page=100
https://auto.ru/moskva/cars/bmw/used/?page=101
https://auto.ru/moskva/cars/bmw/used/?page=102
https://auto.ru/moskva/cars/bmw/used/?page=103
https://auto.ru/moskva/cars/bmw/used/?page=104
https://auto.ru/moskva/cars/bmw/used/?page=105
https://auto.ru/moskva/cars/bmw/used/?page=106
https://auto.ru/moskva/cars/bmw/used/?page=107
https://auto.ru/moskva/cars/bmw/used/?page=108
https://auto.ru/moskva/cars/bmw/used/?page=109
https://auto.ru/moskva/cars/bmw/used/?page=110
https://auto.ru/moskva/cars/bmw/used/?page=111
https://auto.ru/moskva/cars/bmw/used/?page=112
https://auto.ru/moskva/cars/bmw/used/?page=113
https://auto.ru/moskva/cars/bmw/used/?page=114
https://auto.ru/moskva/cars/bmw/used/?page=115
https://auto.ru/moskva/cars/bmw/used/?page=116
https://auto.ru/moskva/cars/bmw/used/?page=117
https://auto.ru/moskva/cars/bmw/used/?page=118
https://auto.ru

https://auto.ru/moskva/cars/volkswagen/used/?page=108
https://auto.ru/moskva/cars/volkswagen/used/?page=109
https://auto.ru/moskva/cars/volkswagen/used/?page=110
https://auto.ru/moskva/cars/volkswagen/used/?page=111
https://auto.ru/moskva/cars/volkswagen/used/?page=112
https://auto.ru/moskva/cars/volkswagen/used/?page=113
https://auto.ru/moskva/cars/volkswagen/used/?page=114
https://auto.ru/moskva/cars/volkswagen/used/?page=115
https://auto.ru/moskva/cars/volkswagen/used/?page=116
https://auto.ru/moskva/cars/volkswagen/used/?page=117
https://auto.ru/moskva/cars/volkswagen/used/?page=118
https://auto.ru/moskva/cars/volkswagen/used/?page=119
https://auto.ru/moskva/cars/volkswagen/used/?page=120
https://auto.ru/moskva/cars/volkswagen/used/?page=121
VOLKSWAGEN 4511
2765
https://auto.ru/moskva/cars/audi/used/?page=2
https://auto.ru/moskva/cars/audi/used/?page=3
https://auto.ru/moskva/cars/audi/used/?page=4
https://auto.ru/moskva/cars/audi/used/?page=5
https://auto.ru/moskva/cars/audi/used/?

https://auto.ru/moskva/cars/toyota/used/?page=83
https://auto.ru/moskva/cars/toyota/used/?page=84
https://auto.ru/moskva/cars/toyota/used/?page=85
https://auto.ru/moskva/cars/toyota/used/?page=86
https://auto.ru/moskva/cars/toyota/used/?page=87
https://auto.ru/moskva/cars/toyota/used/?page=88
https://auto.ru/moskva/cars/toyota/used/?page=89
https://auto.ru/moskva/cars/toyota/used/?page=90
https://auto.ru/moskva/cars/toyota/used/?page=91
https://auto.ru/moskva/cars/toyota/used/?page=92
https://auto.ru/moskva/cars/toyota/used/?page=93
https://auto.ru/moskva/cars/toyota/used/?page=94
https://auto.ru/moskva/cars/toyota/used/?page=95
https://auto.ru/moskva/cars/toyota/used/?page=96
https://auto.ru/moskva/cars/toyota/used/?page=97
https://auto.ru/moskva/cars/toyota/used/?page=98
https://auto.ru/moskva/cars/toyota/used/?page=99
https://auto.ru/moskva/cars/toyota/used/?page=100
https://auto.ru/moskva/cars/toyota/used/?page=101
https://auto.ru/moskva/cars/toyota/used/?page=102
https://auto.ru/m

In [180]:
save_data("collected_links_Moscow.json", offers_by_brand)

In [22]:
offers_by_brand = load_data("collected_links_May_10.json")

# Информация по предложениям

In [77]:
def collect_car_data(car_url,brand):     # Собирает информацию по машинам
    
    #print(car_url) ##
    
    car_features={}                 # словарь для сбора информации по одной машине
    
    # Бренд 
    car_features['brand']=brand            

    car_info = requests.get(car_url)                                #,headers={'User-Agent': UserAgent().chrome})
    soup_by_car = bs(car_info.content)
    
    try:
        card_info = soup_by_car.find(class_='CardInfo')        
        more_car_info = card_info.find_all(class_='Link Link_color_black')
        for ind, more_car in enumerate(more_car_info):
            car_features[(str(ind)+'more')]=more_car.get_text()                                                   
            #print()                      ##
    except Exception as e8:     
        print(car_url, brand,'Fuel')
        print(e8)
      
    try:   #Модель                          
        header = soup_by_car.find(class_='CardSidebarActions__title').get_text()
        car_features['model']=header
        #print(header.prettify())    ##
    except Exception as e2:     
        print(car_url, brand, 'car_model')
        print(e2)
        
    try:  # Другие характеристики машины
        for li in card_info.find_all('li'):
            feature=li.get_text("|",strip=True).replace("\xa0", " ") 
            car_features[feature.split("|")[0].strip()]=feature.split("|")[1].strip()
        #print(car_features)           ##
    except Exception as e3:     
        print(car_url, brand, 'feature')
        print(e3)
    
    try:
        # ЦЕНА
        price = soup_by_car.find(class_='OfferPriceCaption__price').get_text("|", strip=True)[:-2]
        car_features['price']="".join(price.split())
        #print(car_features)           ##
    except Exception as e4:     
        print(car_url, brand, 'price')
        print(e4)     
        car_features['price']="SOLD"
        
       
    return(car_features)

In [78]:
car_link='https://auto.ru/cars/used/sale/bmw/x6/1101324094-b44cbb00/' # TEST
car_brand='BMW'
collect_car_data(car_link,car_brand)

{'brand': 'BMW',
 '0more': '2018',
 '1more': 'внедорожник 5 дв.',
 '2more': 'чёрный',
 '3more': 'Дизель',
 'model': 'BMW X6 II (F16)',
 'год выпуска': '2018',
 'Пробег': '21 275 км',
 'Кузов': 'внедорожник 5 дв.',
 'Цвет': 'чёрный',
 'Двигатель': '3.0 л / 381 л.с. /',
 'Налог': '114 300 ₽ / год',
 'Коробка': 'автоматическая',
 'Привод': 'полный',
 'Руль': 'Левый',
 'Состояние': 'Не требует ремонта',
 'Владельцы': '1 владелец',
 'ПТС': 'Оригинал',
 'Владение': '2 года и 10 месяцев',
 'Таможня': 'Растаможен',
 'Гарантия': 'До октября 2021',
 'VIN': 'WBA**************',
 'Госномер': '******',
 'price': '5900000'}

# Action2

In [28]:
links = load_data("collected_links_May_10.json") 

set_links=set()                    # Посмотрим количество собранных уникальных ссылок

for brand in brands:
    for link in links[brand]:
        
        set_links.add(link)
len(set_links)

41083

In [79]:
collected_data=[]
links = load_data("collected_links_May_10.json")

In [80]:
for brand in ['SKODA','VOLVO', 'MITSUBISHI', 'BMW']:  #Остатки сладки
    print(brand)
    stop=5000
    if len(links[brand])<5000:
        stop=len(links[brand])
        
    for car_url in (links[brand][0:(stop-1)]):
        
        collect=collect_car_data(car_url,brand)
        
        collected_data.append(collect)
        
print(len(collected_data))

SKODA
https://auto.ru/cars/used/sale/skoda/octavia/1102606028-a42b8bdd/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102606028-a42b8bdd/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/kodiaq/1101882516-37dbaacb/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/kodiaq/1101882516-37dbaacb/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102828191-05a9f1a1/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102828191-05a9f1a1/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103491815-aff1d3a6/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103491815-aff1d3a6/ SKODA price
'NoneType' object has no attribute

https://auto.ru/cars/used/sale/skoda/octavia/1103454536-7e6b2b79/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103454536-7e6b2b79/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102374412-2695936e/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102374412-2695936e/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103453946-36772d26/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103453946-36772d26/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/fabia/1103474045-443252d3/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/fabia/1103474045-443252d3/ SKODA price
'NoneType' object has no attribute 'get_te

https://auto.ru/cars/used/sale/skoda/octavia/1102660477-dcc1dc21/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102660477-dcc1dc21/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/yeti/1102832152-985312c3/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/yeti/1102832152-985312c3/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/rapid/1091105678-b042f6c0/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/rapid/1091105678-b042f6c0/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/yeti/1102911106-8f78016b/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/yeti/1102911106-8f78016b/ SKODA price
'NoneType' object has no attribute 'get_text'
https://

https://auto.ru/cars/used/sale/skoda/fabia/1102424220-dbe196bf/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/fabia/1102424220-dbe196bf/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102304536-6f8c468a/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102304536-6f8c468a/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102321141-a6a206fc/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102321141-a6a206fc/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1101690301-322474bf/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1101690301-322474bf/ SKODA price
'NoneType' object has no attribute 'get_te

https://auto.ru/cars/used/sale/skoda/octavia/1102828191-05a9f1a1/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102828191-05a9f1a1/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103491815-aff1d3a6/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1103491815-aff1d3a6/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/kodiaq/1101882516-37dbaacb/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/kodiaq/1101882516-37dbaacb/ SKODA price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102606028-a42b8bdd/ SKODA car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/skoda/octavia/1102606028-a42b8bdd/ SKODA price
'NoneType' object has no attribute 'get_

...

https://auto.ru/cars/used/sale/bmw/7er/1102496120-3bd815dc/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1102496120-3bd815dc/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/3er/1101908959-921311a4/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/3er/1101908959-921311a4/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x6/1103477322-ec20572a/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x6/1103477322-ec20572a/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1103471247-688df39c/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1103471247-688df39c/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1103113902-4fd5cd1a

https://auto.ru/cars/used/sale/bmw/x5/1103165823-6030eccf/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103165823-6030eccf/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103165823-6030eccf/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103165823-6030eccf/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x4/1103344060-290ef156/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x4/1103344060-290ef156/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1103410166-86c52f98/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1103410166-86c52f98/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/4/1103065466-62a4db79/ BMW 

https://auto.ru/cars/used/sale/bmw/x6/1103417951-1fc4f038/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x6/1103417951-1fc4f038/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x3/1103478067-5d03fa80/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x3/1103478067-5d03fa80/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1102888026-ac212b64/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1102888026-ac212b64/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/2grandtourer/1101426527-6dbd3d1d/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/2grandtourer/1101426527-6dbd3d1d/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/11016

https://auto.ru/cars/used/sale/bmw/5er/1101958646-ccc96b0b/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1101958646-ccc96b0b/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1101767963-66a555c7/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1101767963-66a555c7/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/5er/1101958646-ccc96b0b/ 

https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103468506-bedde903/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103468506-bedde903/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1101767963-66a555c7/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1101767963-66a555c7/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW car_model
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/x5/1103493936-4b370813/ BMW price
'NoneType' object has no attribute 'get_text'
https://auto.ru/cars/used/sale/bmw/7er/1101767963-66a555c7/ BM

In [81]:
# преобразование в датафрейм

cars_df=pd.DataFrame.from_records(collected_data)
display(cars_df)

,brand,0more,1more,2more,3more,model,год выпуска,Пробег,Кузов,Цвет,...,ПТС,Таможня,VIN,Госномер,price,Владение,Обмен,Гарантия,Кузов №,Запас хода
0,SKODA,2012,лифтбек,серебристый,Бензин,Skoda Octavia II (A5) Рестайлинг,2012,140 000 км,лифтбек,серебристый,...,Оригинал,Растаможен,XW8**************,******,600000,NaN,NaN,NaN,NaN,NaN
1,SKODA,2007,универсал 5 дв.,серый,Бензин,Skoda Octavia I (A4) Рестайлинг,2007,197 000 км,универсал 5 дв.,серый,...,Оригинал,Растаможен,TMB**************,******,265000,4 года и 6 месяцев,NaN,NaN,NaN,NaN
2,SKODA,2018,лифтбек,серебристый,Бензин,Skoda Octavia III (A7) Рестайлинг,2018,27 000 км,лифтбек,серебристый,...,Оригинал,Растаможен,XW8**************,******,1505000,NaN,NaN,NaN,NaN,NaN
3,SKODA,2015,внедорожник 5 дв.,белый,Бензин,Skoda Yeti I Рестайлинг,2015,77 000 км,внедорожник 5 дв.,белый,...,Оригинал,Растаможен,XW8**************,NaN,940000,NaN,Рассмотрю варианты,NaN,NaN,NaN
4,SKODA,2016,внедорожник 5 дв.,бежевый,Бензин,Skoda Yeti I Рестайлинг,2016,82 000 км,внедорожник 5 дв.,бежевый,...,Оригинал,Растаможен,XW8**************,NaN,955000,NaN,Рассмотрю варианты,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17811,BMW,2016,хэтчбек 5 дв.,белый,Бензин,BMW 1 серии II (F20/F21) Рестайлинг,2016,100 530 км,хэтчбек 5 дв.,белый,...,Оригинал,Растаможен,WBA**************,******,1250000,5 лет и 2 месяца,Рассмотрю варианты,NaN,NaN,NaN
17812,BMW,2017,седан,серый,Дизель,BMW 3 серии VI (F3x) Рестайлинг,2017,97 000 км,седан,серый,...,Оригинал,Растаможен,X4X**************,NaN,1895000,NaN,Рассмотрю варианты,NaN,NaN,NaN
17813,BMW,2013,внедорожник 5 дв.,серебристый,Дизель,BMW X3 II (F25),2013,44 566 км,внедорожник 5 дв.,серебристый,...,Оригинал,Растаможен,X4X**************,******,2000000,7 лет и 5 месяцев,NaN,NaN,NaN,NaN
17814,BMW,2019,внедорожник 5 дв.,бежевый,Дизель,BMW X7 I (G07),2019,22 500 км,внедорожник 5 дв.,бежевый,...,Оригинал,Растаможен,WBA**************,******,7777000,NaN,Рассмотрю варианты,До января 2025,NaN,NaN


In [82]:
save_data("collected_data_set_SCMB.json", collected_data) #Save the collected data in JSON

In [83]:
cars_df.to_csv('train_p5_full_SMW.csv',index=False)  # Save the dataframe

In [ ]:
collected_data = load_data("collected_data_set.json")